In [38]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [39]:
url = 'https://sofifa.com/players?type=all&oal=75&vlh=0'
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")
html.status_code

200

In [40]:
productos = soup.find_all("td", {"class": "col-name"})

In [41]:
jugadores = {'jugador': []}

In [42]:
for i in productos:
    jugadores['jugador'].append(i.find('a').get('href'))  

In [43]:
jugadores_top = pd.DataFrame(jugadores)
jugadores_top.head(2)

,jugador
0,/player/184087/toby-alderweireld/220057/
1,/players?na=7


In [44]:
jugadores_top['jug'] = jugadores_top['jugador'].str.extract(r'(?<=\/player\/)(.*)(?=\/220057)')

In [45]:
jugadores_top.head(2)

,jugador,jug
0,/player/184087/toby-alderweireld/220057/,184087/toby-alderweireld
1,/players?na=7,NaN


In [46]:
jugadores_fichar = jugadores_top.dropna()
jugadores_fichar = jugadores_fichar.reset_index(drop=True)
jugadores_fichar.head(2)

,jugador,jug
0,/player/184087/toby-alderweireld/220057/,184087/toby-alderweireld
1,/player/257536/jan-kuchta/220057/,257536/jan-kuchta


In [47]:
equipos = jugadores_fichar.drop(['jugador'], axis=1)
equipos

,jug
0,184087/toby-alderweireld
1,257536/jan-kuchta
2,205211/milan-borjan
3,188044/lan-a
4,189060/aleksandar-dragovic
5,236298/eduard-atuesta
6,186347/maciej-rybus


In [48]:
jugar_caract = pd.DataFrame()
for i in range(0,7):
    jug = equipos['jug'][i]
    
    
    url2 = 'https://sofifa.com/player/' + jug + '/220057/'
    html2 = requests.get(url2)
    soup2 = BeautifulSoup(html2.content, "html.parser")
    html2.status_code
    
    productos2 = soup2.find_all("ul", {"class": "pl"})
    
    
    caract = {'jugador': []}
    for i in productos2:
        caract['jugador'].append(i) 
    
    caract = pd.DataFrame(caract)

    caract.rename(columns = {'jugador': jug}, inplace = True)
    caract[jug] = caract[jug].astype(str)
    
    caract[jug] = caract[jug].str.extract(r'(?<=\d\"\>)(\d+)(?=\<\/span\>)')
    
    caract_jug = caract.dropna()
    caract_jug = caract_jug.T

    
    jugar_caract = pd.concat([jugar_caract, caract_jug], axis=0)

In [49]:
jugar_caract = jugar_caract.reset_index(drop=True)

In [50]:
jugar_caract.head(2)

,2,3,4,5,6,7,8,9,10
0,83,83,64,62,55,78,79,87,16
1,77,79,59,72,79,75,87,27,10


In [51]:
nuevas_columnas = {
    2 : 'Valoración equipo actual', 
    3: "Mejor valoración individual", 
    4: "Ataque", 
    5: "Regate", 
    6: "Aceleración",
    7: 'Potencia tiro',
    8: 'Agresividad',
    9: 'Defensa',
    10: 'Portería'}

In [52]:
jugar_caract.rename(columns = nuevas_columnas, inplace = True)
jugar_caract.head()

,Valoración equipo actual,Mejor valoración individual,Ataque,Regate,Aceleración,Potencia tiro,Agresividad,Defensa,Portería
0,83,83,64,62,55,78,79,87,16
1,77,79,59,72,79,75,87,27,10
2,73,77,15,28,47,56,44,21,78
3,69,77,66,83,84,73,55,35,11
4,77,76,53,62,62,69,78,77,10


In [53]:
jugar_caract.to_csv('../../data/veteranos/valoraciones_jugadores.csv')